<a href="https://colab.research.google.com/github/TitasDas/CNN-feature-extraction/blob/main/transfer_learning_vgg19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pathlib
import os
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import pickle

import tensorflow as tf
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input 

In [2]:
pwd

'/content'

In [3]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
input_images = []
output_feature_vectors = {}

with os.scandir(path = '/content/drive/MyDrive/Dataset/cristiano') as files:
  for file in files:
    if file.name.endswith('.jpg'):
      input_images.append(file)
print(len(input_images))

455


In [5]:
def features_extraction(file, model):
  org_image = image.load_img(file, target_size=(224,224))
  nparr_image = image.img_to_array(org_image)
  reshaped_im = np.expand_dims(nparr_image, axis=0)
  img_for_model = preprocess_input(reshaped_im)
  features = model.predict(img_for_model)
  return features

In [24]:
base_model = VGG19(weights = 'imagenet')
model = Model(inputs = base_model.input, outputs = base_model.layers[-2].output)
#block1_conv1,block1_conv2,block1_pool
#block2_conv1-2,block2_pool
#block3_conv1-4,block3_pool
#block4_conv1-4,block4_pool
#block5_conv1-4,block4_pool


In [25]:
output_feature_vectors = {im.name:features_extraction(im,model) for im in input_images }
#print(output_feature_vectors)

In [32]:
image_file_names = np.array(list(output_feature_vectors.keys()))
only_features = np.array(list(output_feature_vectors.values()))
#only_features.shape
only_features = only_features.reshape(-1,4096)

In [43]:
#choosing n components and using svd for dimensionality reduction
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline

total_variance = 0
calc_n_comp = 0
svd = TruncatedSVD(n_components = only_features.shape[1]-1,n_iter = 8)
dummy_reduced_matrix = svd.fit(only_features)
variance_ratio = svd.explained_variance_ratio_
for explained_variance in variance_ratio:
  total_variance += explained_variance
  calc_n_comp += 1
  if total_variance >= 0.95:
    break
final_n_comp = calc_n_comp


svd = TruncatedSVD(n_components=final_n_comp, n_iter=8)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)
reduced_feature_matrix = lsa.fit_transform(only_features)


In [46]:
#find optimum no of clusters
from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import KMeans
from kneed import KneeLocator

max_clusters = only_features.shape[0]
iters = range(2,max_clusters + 1,2)
sum_squared_distances = []
for k in iters:
  sum_squared_distances.append(MiniBatchKMeans(n_clusters = k, init_size = 1 , batch_size = 5, random_state = 2).fit(reduced_feature_matrix).inertia_)
kneedle = KneeLocator(iters, sum_squared_distances, S=1.0, curve="convex", direction = "decreasing")
if kneedle.knee is not None:
  num_clusters = round(kneedle.knee, 3)
else:
  num_clusters = max_clusters

/usr/local/lib/python3.7/dist-packages/sklearn/cluster/_kmeans.py:1576: RuntimeWarning: init_size=1 should be larger than k=2. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.7/dist-packages/sklearn/cluster/_kmeans.py:1576: RuntimeWarning: init_size=1 should be larger than k=2. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.7/dist-packages/sklearn/cluster/_kmeans.py:1576: RuntimeWarning: init_size=1 should be larger than k=2. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.7/dist-packages/sklearn/cluster/_kmeans.py:1576: RuntimeWarning: init_size=1 should be larger than k=4. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.7/dist-packages/sklearn/cluster/_kmeans.py:1576: RuntimeWarning: init_size=1 should be larger than k=4. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.7/dist-packages/sklearn/cluster/_kmeans.py:1576: RuntimeWarning: init_size=1 should be larger than k=4. Setting it to 3*k
  init_size=ini